In [2]:
import numpy as np
import pandas as pd
import time
import datetime
from __future__ import print_function

In [4]:
folder = '../'
data = pd.read_csv(folder + 'Austin_B-Cycle_Trips.csv')

In [5]:
data.head()

,Trip ID,Membership Type,Bicycle ID,Checkout Date,Checkout Time,Checkout Kiosk ID,Checkout Kiosk,Return Kiosk ID,Return Kiosk,Trip Duration Minutes,Month,Year
0,9900285854,Annual (San Antonio B-cycle),207.0,10/26/2014,13:12:00,2537.0,West & 6th St.,2707.0,Rainey St @ Cummings,76,10.0,2014.0
1,9900285855,24-Hour Kiosk (Austin B-cycle),969.0,10/26/2014,13:12:00,2498.0,Convention Center / 4th St. @ MetroRail,2566.0,Pfluger Bridge @ W 2nd Street,58,10.0,2014.0
2,9900285856,Annual Membership (Austin B-cycle),214.0,10/26/2014,13:12:00,2537.0,West & 6th St.,2496.0,8th & Congress,8,10.0,2014.0
3,9900285857,24-Hour Kiosk (Austin B-cycle),745.0,10/26/2014,13:12:00,NaN,Zilker Park at Barton Springs & William Barton...,NaN,Zilker Park at Barton Springs & William Barton...,28,10.0,2014.0
4,9900285858,24-Hour Kiosk (Austin B-cycle),164.0,10/26/2014,13:12:00,2538.0,Bullock Museum @ Congress & MLK,NaN,Convention Center/ 3rd & Trinity,15,10.0,2014.0


In [7]:
#CONVERT TIME TO EPOCH TIME AND REORDER DATA BY TIME
pattern = '%m/%d/%Y %H:%M:%S'

def getDay(row):
    return int(row["Checkout Date"].split('/')[1])

def getMonth(row):
    return int(row["Checkout Date"].split('/')[0])

def getYear(row):
    return int(row["Checkout Date"].split('/')[2])

#MONDAY=0, SUNDAY=6
def getDayOfWeek(row):
    month, day, year = [int(x) for x in row["Checkout Date"].split('/')]
    return datetime.date(year, month, day).weekday()
    
def getEpochTime(row):
    date_time = row["Checkout Date"] + ' ' +row["Checkout Time"]
    epoch = int(time.mktime(time.strptime(date_time, pattern)))
    return epoch

def getRetEpochTime(row):
    return int(row['Checkout Epoch Time']) + int(row['Trip Duration Minutes'])*60

for index, row in data.iterrows():
    if np.isnan(row['Month']):
        data.at[index, 'Month'] = row["Checkout Date"].split('/')[0]
    if np.isnan(row['Year']):
        data.at[index, 'Year'] = row["Checkout Date"].split('/')[2]
        
data['Day'] = data.apply(getDay, axis=1)
data['Month'] = data.apply(getMonth, axis=1)
data['Year'] = data.apply(getYear, axis=1)
data['Day of Week'] = data.apply(getDayOfWeek, axis=1)
data['Checkout Epoch Time'] = data.apply(getEpochTime,axis=1)
data['Return Epoch Time'] = data.apply(getRetEpochTime, axis=1)
del data['Checkout Date']
data = data.sort_values('Checkout Epoch Time', axis=0)
data = data.reset_index(drop=True)

In [8]:
data.head()

,Trip ID,Membership Type,Bicycle ID,Checkout Time,Checkout Kiosk ID,Checkout Kiosk,Return Kiosk ID,Return Kiosk,Trip Duration Minutes,Month,Year,Day,Day of Week,Checkout Epoch Time,Return Epoch Time
0,9900333979,Founding Member (Austin B-cycle),966.0,9:12:00,2495.0,4th & Congress,2500.0,Republic Square,5,12,2013,21,5,1387638720,1387639020
1,9900333980,Annual Membership (Austin B-cycle),36.0,9:12:00,2496.0,8th & Congress,2497.0,Capitol Station / Congress & 11th,5,12,2013,21,5,1387638720,1387639020
2,9900333981,24-Hour Kiosk (Austin B-cycle),101.0,10:12:00,2500.0,Republic Square,2503.0,South Congress & James,19,12,2013,21,5,1387642320,1387643460
3,9900333982,24-Hour Kiosk (Austin B-cycle),252.0,13:12:00,2497.0,Capitol Station / Congress & 11th,2497.0,Capitol Station / Congress & 11th,4,12,2013,21,5,1387653120,1387653360
4,9900333983,24-Hour Kiosk (Austin B-cycle),287.0,13:12:00,2501.0,5th & Bowie,2495.0,4th & Congress,12,12,2013,21,5,1387653120,1387653840


In [9]:
outputFolder = '../metadata/'
data.to_csv(outputFolder + 'Austin_B-Cycle_Trips_extended_ordered.csv')

In [11]:
#LOOK AT DUPLICATE/INCONSISTENT KIOSK IDS
kioskIds = dict()
for index, row in data.iterrows():
    if row['Checkout Kiosk'] not in kioskIds:
        kioskIds[row['Checkout Kiosk']] = set()
    if np.isnan(row['Checkout Kiosk ID']):
        kioskIds[row['Checkout Kiosk']].add(-1)
    else:
        kioskIds[row['Checkout Kiosk']].add(row['Checkout Kiosk ID'])
    
    if row['Return Kiosk'] not in kioskIds:
        kioskIds[row['Return Kiosk']] = set()
    if np.isnan(row['Return Kiosk ID']):
        kioskIds[row['Return Kiosk']].add(-1)
    else:
        kioskIds[row['Return Kiosk']].add(row['Return Kiosk ID'])
        
for key in kioskIds:
    if len(kioskIds[key]) == 1 and not list(kioskIds[key])[0] == -1:
        continue
        
    print(str(key) + ":   ", end='')
    for val in kioskIds[key]:
            print(str(val) + " ", end='')
    print()

Re-branding:   -1 
MapJam at French Legation:   -1 
Main Office:   1001.0 -1 
State Capitol @ 14th & Colorado:   2541.0 -1 
Lavaca & 6th:   3294.0 1007.0 
Dean Keeton & Speedway :   3794.0 -1 
Shop:   -1 
Mobile Station @ Bike Fest:   -1 
Earth Day ATX 2017:   -1 
Main Shop:   -1 
Stolen:   1001.0 -1 
Zilker Park at Barton Springs & William Barton Drive:   -1 
Red River @ LBJ Library:   -1 
Mobile Station:   -1 
Customer Service:   -1 
Repair Shop:   -1 
ACC - West & 12th:   -1 
Mobile Station @ Unplugged:   -1 
Eeyore's 2018:   -1 
MapJam at Pan Am Park:   -1 
Eeyore's 2017:   -1 
Mobile Station @ Boardwalk Opening Ceremony:   -1 
Bullock Museum @ Congress & MLK:   2538.0 -1 
Republic Square @ 5th & Guadalupe:   3456.0 3455.0 
Fantasy Zilker:   -1 
Missing:   -1 
Marketing Event:   -1 
MapJam at Hops & Grain Brewery:   -1 
Convention Center/ 3rd & Trinity:   -1 
East 11th Street at Victory Grill:   -1 
MapJam at Scoot Inn:   -1 


In [12]:
#CREATE DICTIONARY OF KIOSK NAME/NUMBER
fixKiosk = dict()
#HARDCODE TO FIX CONFLICTS
fixKiosk['Republic Square @ Federal Courthouse Plaza'] = 3455
fixKiosk['Republic Square @ 5th & Guadalupe'] = 3456
fixKiosk['Lavaca & 6th'] = 1027
fixKiosk['Guadalupe & 6th'] = 3294
fixKiosk['Stolen'] = 1000
fixKiosk['Main Office'] = 1001

#ASSIGN MIXED NAN VALUES THEIR KIOSK ID
nanId = 1
for key in kioskIds:
    if key in fixKiosk: #skip hardcoded ones
        continue
        
    if len(kioskIds[key]) == 1 and list(kioskIds[key])[0] == -1: #only nan
        fixKiosk[key] = nanId
        nanId += 1
        continue
        
    for futureId in list(kioskIds[key]):
        if futureId != -1:
            fixKiosk[key] = futureId

In [87]:
#LABEL UNLABLLED BIKE
for index, row in data.iterrows():
    if np.isnan(row['Bicycle ID']):
        data.at[index, 'Bicycle ID'] = 0
print(len(data[data['Bicycle ID'] == 0]))

723


In [17]:
#APPLY CONSISTENT KIOSKID LABELS
for index, row in data.iterrows():
    if fixKiosk[row['Checkout Kiosk']] != row['Checkout Kiosk ID']:
        data.at[index, 'Checkout Kiosk ID'] = fixKiosk[row['Checkout Kiosk']]
    if fixKiosk[row['Return Kiosk']] != row['Return Kiosk ID']:
        data.at[index, 'Return Kiosk ID'] = fixKiosk[row['Return Kiosk']]

In [19]:
print(data.head())
outputFolder = '../metadata/'
data.to_csv(outputFolder + 'Austin_B-Cycle_Trips_extended_ordered.csv')

      Trip ID                     Membership Type  Bicycle ID Checkout Time  \
0  9900333979    Founding Member (Austin B-cycle)       966.0       9:12:00   
1  9900333980  Annual Membership (Austin B-cycle)        36.0       9:12:00   
2  9900333981      24-Hour Kiosk (Austin B-cycle)       101.0      10:12:00   
3  9900333982      24-Hour Kiosk (Austin B-cycle)       252.0      13:12:00   
4  9900333983      24-Hour Kiosk (Austin B-cycle)       287.0      13:12:00   

   Checkout Kiosk ID                     Checkout Kiosk  Return Kiosk ID  \
0             2495.0                     4th & Congress           2500.0   
1             2496.0                     8th & Congress           2497.0   
2             2500.0                    Republic Square           2503.0   
3             2497.0  Capitol Station / Congress & 11th           2497.0   
4             2501.0                        5th & Bowie           2495.0   

                        Return Kiosk  Trip Duration Minutes  Month  

In [20]:
print(data.iloc[732745:,:].head())
dataFolder = '../metadata/'
data.iloc[732745:,:].to_csv(dataFolder + 'Trips_After_Student_Membership.csv')

         Trip ID          Membership Type  Bicycle ID Checkout Time  \
732745  16653608  U.T. Student Membership       434.0       7:20:10   
732746  16653609  U.T. Student Membership       472.0       7:21:04   
732747  16653610  U.T. Student Membership       277.0       7:21:37   
732748  16653687                  Walk Up       822.0       9:07:57   
732749  16653714                  Walk Up       465.0       9:27:20   

        Checkout Kiosk ID              Checkout Kiosk  Return Kiosk ID  \
732745             3792.0                22nd & Pearl           3798.0   
732746             3792.0                22nd & Pearl           3798.0   
732747             3792.0                22nd & Pearl           3798.0   
732748             2562.0    San Jacinto & 8th Street           2548.0   
732749             3686.0  Sterzing at Barton Springs           2574.0   

                    Return Kiosk  Trip Duration Minutes  Month  Year  Day  \
732745      21st & Speedway @PCL                   

In [92]:
print("Number of unlabeled membership transactions: {}".format(data.isnull().values.ravel().sum()))

Number of unlabeled membership transactions: 6250
